First, import all the necessary libraries:
* liblibra_core - for general data types from Libra
* libra_py - for the normal modes module
* py3Dmol - for visualization

The output of the cell below will throw a bunch of warnings, but this is not a problem nothing really serios. So just disregard them.

In [1]:
import os
import sys
import math
from liblibra_core import *
from libra_py import *
import math
import py3Dmol

/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::complex<double>, std::allocator<std::complex<double> > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, s

Load a Universe - it contains the properties of all elements. The third argument (set to 0) in the *Load_PT* function disables the verbose output

In [2]:
U = Universe(); LoadPT.Load_PT(U, os.getcwd()+"/elements.dat", 0)

Create the system and populate it

In [3]:
syst = System()
LoadMolecule.Load_Molecule(U, syst, os.getcwd()+"/azulene.pdb", "iqmol_pdb", 0)

40


In [4]:
print( "Number of atoms in the system = ", syst.Number_of_atoms)
atlst1 = range(0,syst.Number_of_atoms)
print( syst.get_xyz(0, "abc", 0))

Number of atoms in the system =  18
18
Molecule frame= 0 
C -2.486 -0.69 2.097
C -2.635 -0.295 0.77
C -2.72 1.007 0.306
C -2.679 2.255 1.042
C -2.544 2.472 2.403
C -2.415 1.494 3.386
C -2.39 0.102 3.247
C -2.867 1.335 -1.03
C -2.918 2.711 -1.144
C -2.804 3.272 0.114
H -2.439 -1.76 2.262
H -2.689 -1.089 0.033
H -2.538 3.502 2.743
H -2.32 1.863 4.4
H -2.278 -0.449 4.173
H -2.933 0.636 -1.854
H -3.03 3.261 -2.069
H -2.812 4.333 0.329



Do electronic structure calculations:

In [5]:
lstHamQM = listHamiltonian_QM("control_parameters_eht.dat", syst)
lstHamQM.compute_scf(syst)
el_str = lstHamQM.get_electronic_structure()

Print out the orbital energies

In [6]:
print( "Bands(alp)    Occupations(alp)       Bands(bet)    Occupations(bet)")
for j in range(el_str.Norb):
    print( "%12.8f   %12.8f  %12.8f   %12.8f" %(el_str.get_bands_alp(j), el_str.get_occ_alp(j), el_str.get_bands_bet(j), el_str.get_occ_bet(j) ) )


Bands(alp)    Occupations(alp)       Bands(bet)    Occupations(bet)
 -1.01578066     1.00000000   -1.01578066     1.00000000
 -0.95173220     1.00000000   -0.95173220     1.00000000
 -0.91362212     1.00000000   -0.91362212     1.00000000
 -0.84812471     1.00000000   -0.84812471     1.00000000
 -0.81195363     1.00000000   -0.81195363     1.00000000
 -0.73379578     1.00000000   -0.73379578     1.00000000
 -0.70963007     1.00000000   -0.70963007     1.00000000
 -0.65893718     1.00000000   -0.65893718     1.00000000
 -0.62818704     1.00000000   -0.62818704     1.00000000
 -0.59896158     1.00000000   -0.59896158     1.00000000
 -0.59293944     1.00000000   -0.59293944     1.00000000
 -0.56224804     1.00000000   -0.56224804     1.00000000
 -0.55643719     1.00000000   -0.55643719     1.00000000
 -0.54902929     1.00000000   -0.54902929     1.00000000
 -0.51787565     1.00000000   -0.51787565     1.00000000
 -0.50133128     1.00000000   -0.50133128     1.00000000
 -0.49410833     1.0

Compute the charge density for HOMO and LUMO

In [7]:
# Compute homo index
os.system("mkdir char_dens")
homo = el_str.Nocc_alp - 1   # index of the HOMO orbital

prms = Control_Parameters()
prms.orbs = Py2Cpp_int([homo, homo+1]) 
prms.nx_grid, prms.ny_grid, prms.nz_grid = 40, 40, 40
prms.charge_density_prefix = "char_dens/"

charge_density( el_str, syst, lstHamQM.basis_ao, prms)

In [9]:
# Method 1
xyz = syst.get_xyz(0, "abc", 0)

view = py3Dmol.view(width=800,height=400)
view.addModel(xyz,'xyz',{'vibrate': {'frames':10,'amplitude':1}})
#view.setStyle({'sphere':{}})
view.setBackgroundColor('0xeeeeee')


bohr = open('char_dens/_orbital_23.cube').read()
view.addVolumetricData(bohr, "cube", {'isoval':  0.01, 'color': "red", 'opacity': 0.95})
view.addVolumetricData(bohr, "cube", {'isoval': -0.01, 'color': "blue", 'opacity': 0.95})

view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol